# MOUNT The Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Preprocessing

In [0]:
def preprocessing(filename):
  video_path = filename

  img_arr=[]
  cap = cv2.VideoCapture(video_path)
  success, img = cap.read()
  idx = 0
  num=1
  while success:
    idx+=1
    if num==idx:
      img_arr.append(img)
      num+=6
    success, img = cap.read()

  height, width, layers = img_arr[0].shape
  size = (width,height)

  out = cv2.VideoWriter('preprocess.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

  for i in range(len(img_arr)):
      out.write(img_arr[i])

  out.release()
  cv2.destroyAllWindows()  
  
  result_path = "/content/preprocess.avi"

  return  img_arr, result_path 



# Violence preparation


In [0]:
import sys
import cv2
import os
import numpy as np
import keras
from keras.models import model_from_json
from keras import backend as K



path = "/content/vid2"
os.mkdir(path)


# load json and create model
json_file = open('/content/drive/My Drive/Emergency project/ViolenceDetection/pretrained/VGG16_Edited_model.json', 'r')
loaded_image_model_transfer_json = json_file.read()
json_file.close()
loaded_image_model_transfer = model_from_json(loaded_image_model_transfer_json)
# load weights into new model
loaded_image_model_transfer.load_weights("/content/drive/My Drive/Emergency project/ViolenceDetection/pretrained/VGG16_Edited_model.h5")
print("Loaded model from disk")



# load json and create model
json_file = open('/content/drive/My Drive/Emergency project/ViolenceDetection/pretrained/violence_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/My Drive/Emergency project/ViolenceDetection/pretrained/violence_model.h5")
print("Loaded model from disk")


# # Frame size  
img_size = 224

img_size_touple = (img_size, img_size)

# # Number of channels (RGB)
num_channels = 3


# # classification-layer which is named fc2. This is a fully-connected (or dense) layer.
transfer_layer = loaded_image_model_transfer.get_layer('fc2')

transfer_values_size = K.int_shape(transfer_layer.output)[1]


_images_per_file = 20


def get_frames(current_dir, file_name):
    
    in_file = os.path.join(current_dir, file_name)
    
    images = []
    
    vidcap = cv2.VideoCapture(in_file)
    
    success,image = vidcap.read()
        
    count = 0

    while count<_images_per_file:
                
        RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        res = cv2.resize(RGB_img, dsize=(img_size, img_size),
                                 interpolation=cv2.INTER_CUBIC)
    
        images.append(res)
    
        success,image = vidcap.read()
    
        count += 1
        
    resul = np.array(images)
    
    resul = (resul / 255.).astype(np.float16)
        
    return resul



def get_transfer_values(current_dir, file_name):


    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_touple + (3,)
    
    image_batch = np.zeros(shape=shape, dtype=np.float16)
    
    image_batch = get_frames(current_dir, file_name)
      
    # Pre-allocate output-array for transfer-values.
    # Note that we use 16-bit floating-points to save memory.
    shape = (_images_per_file, transfer_values_size)
    transfer_values = np.zeros(shape=shape, dtype=np.float16)

    transfer_values = \
            loaded_image_model_transfer.predict(image_batch)
            
    return transfer_values



In [0]:
def violence_predict(filename):
  vid_name = filename
  x = get_transfer_values("/content", vid_name)
  x = np.expand_dims(x,axis=0)
  return loaded_model.predict(x)


In [0]:
import os
import shutil

def create_folder_for_packets():  
  shutil.rmtree('vid2')
  path = "/content/vid2"
  os.mkdir(path)
  

# Violence Prediction

In [0]:
import math
from numpy import mean
def fVI(img_arr):

    AVG = []
    img_arr_len = len(img_arr)
    if img_arr_len % 20 != 0: 
        temp_reminder = img_arr_len % 20
        last_frames_packet = img_arr[-20 : ]
        del img_arr[-temp_reminder : ]



    height, width, layers = img_arr[0].shape
    size = (width,height)    

    len_of_packets = math.floor(img_arr_len / 20)
    i = 0
    j = 20

    
    for packet in range(len_of_packets):
        res = img_arr[i : j] 
        i += 20
        j += 20
        out = cv2.VideoWriter('/content/vid2/temp_violence{}.avi'.format(packet),cv2.VideoWriter_fourcc(*'DIVX'), 20 , size)

        for i in range(20):
            out.write(res[i])
    
        out.release()
        cv2.destroyAllWindows()


    for packet in range(len_of_packets):

        violence_prediction = violence_predict('/content/vid2/temp_violence{}.avi'.format(packet))[0][0]
        AVG.append(violence_prediction)
        

    mean_val_of_violence = mean(AVG)

    if mean_val_of_violence > 0.80:
      print("VIOLENCE DETECTED")
    else:
      print("VIOLENCE NOT DETECTED")

    create_folder_for_packets()



# Fire Preparation


In [0]:
!pip install  imageai --upgrade 
!wget install https://github.com/OlafenwaMoses/FireNET/releases/download/v1.0/fire-dataset.zip
!unzip /content/fire-dataset.zip
!wget install https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5
!wget install https://github.com/OlafenwaMoses/FireNET/releases/download/v1.0/detection_model-ex-33--loss-4.97.h5

In [0]:
 %tensorflow_version 1.x
from imageai.Detection.Custom import CustomObjectDetection, CustomVideoObjectDetection
import os

execution_path = os.getcwd()

total_fire_frames = 0
total_non = 0

def forFrame(frame_number, output_array, output_count):
    if bool(output_count):
      global total_fire_frames 
      total_fire_frames  += 1
    else:
      global total_non
      total_non += 1  


def detect_from_video(filename):
    detector = CustomVideoObjectDetection()
    detector.setModelTypeAsYOLOv3()
    detector.setModelPath(detection_model_path=os.path.join(execution_path, "detection_model-ex-33--loss-4.97.h5"))
    detector.setJsonPath(configuration_json=os.path.join(execution_path, "/content/drive/My Drive/Emergency project/FireDetection/detection_config.json"))
    detector.loadModel()
    
    detected_video_path = detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, filename), frames_per_second=30,  minimum_percentage_probability=30, log_progress=False, save_detected_video=False, per_frame_function=forFrame )


# Fire Prediction

In [0]:
def fFI(filename):
  detect_from_video(filename)

  global total_fire_frames
  global total_non

  if (total_fire_frames / (total_fire_frames + total_non)) * 100  >= 15:
    print("FIRE DETECTED")
  else:
    print("NO FIRE DETECTED")  

  
  total_fire_frames -= total_fire_frames
  total_non -= total_non

  





# Accidents Preparation

In [0]:
!wget https://github.com/saifrais/w210-accident-detection/blob/master/inception_v3/retrained_graph.pb

In [0]:

import torch 
from PIL import Image, ImageFile
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torchvision
from PIL import Image, ImageFile
from torch import nn
from torch import optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, models, transforms
from google.colab.patches import cv2_imshow
import cv2

def accidents_predict(filename):

  train_on_gpu = torch.cuda.is_available()
  if not train_on_gpu:
      print('CUDA is not available.  Training on CPU ...')

  # else:
  #     print('CUDA is available!  Training on GPU ...')
  ImageFile.LOAD_TRUNCATED_IMAGES = True
  #!pip install --upgrade wandb




  test_transforms = transforms.Compose([transforms.Resize(255),
                                        #  transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        ])

  model = models.densenet161()


  model.classifier = nn.Sequential(nn.Linear(2208, 1000),
                                  nn.ReLU(),
                                  nn.Dropout(0.2),
                                  nn.Linear(1000, 2),
                                  nn.LogSoftmax(dim=1))

  criterion = nn.NLLLoss()
  # Only train the classifier parameters, feature parameters are frozen
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

  model = model.cuda()

  model.load_state_dict(torch.load("/content/drive/My Drive/Emergency project/AccidentsDetection/tensorboardexp.pt"))
  classes = ["accident", "noaccident"]
  
  count = 0
  counts = 1
  videopath = filename

  label_arr = []
  

  vid = cv2.VideoCapture(videopath)
  ret = True
  while ret:
      if ret == True:
          ret, frame = vid.read()

          try:
              img = Image.fromarray(frame)
          except ValueError:
              break
          except AttributeError:
              break
          img = test_transforms(img)
          img = img.unsqueeze(dim=0)
          img = img.cuda()
          model.eval()
          with torch.no_grad():
              output = model(img)
              _, predicted = torch.max(output, 1)

              index = int(predicted.item())
              if index == 0:
                  count += 1
                  if counts == 1:
                      counts += 1

              labels = classes[index]


          label_arr.append(labels)
          

          if cv2.waitKey(1) & 0xFF == ord('q'):
              break


  
  return label_arr

# Accidents Predict

In [0]:
def fAC(filename):

  total_accident_frames = 0
  total_accident_non = 0

  total_accident = accidents_predict(filename)
  
  for i in total_accident:
    if i == "accident":
      total_accident_frames += 1
    else:
      total_accident_non += 1

  if (total_accident_frames / (total_accident_frames + total_accident_non)) * 100  >= 15:
    print("ACCIDENT DETECTED")
  else:
    print("NO ACCIDENT DETECTED")  


# ALL Emergency Predictions 

In [0]:
from multiprocessing import Process
# Please use the path of the video after uploading it.
video_path = "Your Video Path"
img_arr_for_violence, processed_vid = preprocessing(video_path) 

p1 = Process(target=fFI(processed_vid))
p1.start()

p2 = Process(target=fAC(processed_vid))
p2.start()

p3 = Process(target=fVI(img_arr_for_violence))
p3.start()


p1.join()
p2.join()
p3.join()